In [4]:
# import libraries
from cryptography.fernet import Fernet
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')

In [5]:
# open the key
with open('filekey.key', 'rb') as f:
  key = f.read()

In [6]:
# Store it in fernet
fernet = Fernet(key)

In [7]:
# Open the data
with open('Online Retail.xlsx', 'rb') as f:
  data = f.read()

In [8]:
# Decryption
decrypt_data = fernet.decrypt(data)

In [9]:
# Overwrite the data
with open('Online Retail.xlsx', 'wb') as f:
  f.write(decrypt_data)

In [10]:
# Load the data in pandas Dataframe
df = pd.read_excel('Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [11]:
# Shape of the data
df.shape

(541909, 8)

In [12]:
# Copy of the data
df_copy = df.copy(deep = True)
df_copy.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [13]:
# Create a new column
df_copy['Sales'] = df_copy['Quantity'] * df_copy['UnitPrice']

In [14]:
# Check the columns
df_copy.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Sales'],
      dtype='object')

In [15]:
# Missing Values
df_copy.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0
Sales,0


In [16]:
# Unique Values in Description Column
df_copy['Description'].nunique()

4223

In [18]:
# Clear the white spaces
df_copy['Description'] = df_copy['Description'].str.strip()

In [19]:
# Check again the Unique Values
df_copy['Description'].nunique()

4210

In [21]:
# Price to Description Maping
price_to_description_map = df_copy.dropna(subset=['Description']).drop_duplicates(subset=['UnitPrice'], keep= 'first').set_index('UnitPrice')['Description']

In [22]:
# fill the missing values
df_copy['Description'] = df_copy['Description'].fillna(df_copy['UnitPrice'].map(price_to_description_map))

In [23]:
# Check if Missing Values are replaced
df_copy.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0
Sales,0


In [24]:
# Show the Cancelled Invoices
df_copy[df_copy['InvoiceNo'].astype(str).str.contains('C')].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,-27.50
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,-4.65
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,-19.80
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,-6.96
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,-6.96


In [25]:
# We want to Exclude the cancelled invoices for Market Basket Analysis
df_copy['InvoiceNo'] = df_copy['InvoiceNo'].astype(str)
df_new = df_copy[~df_copy['InvoiceNo'].str.contains('C')]
df_new.shape

(532621, 9)

In [26]:
# Lets explore the new data
df_new.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [27]:
# Unique Countries
df_new['Country'].nunique()

38

In [40]:
# Which Country Gives Us More Sales
df_new.groupby('Country')['Sales'].sum().sort_values(ascending = False).head()

,Sales
Country,
United Kingdom,9003097.964
Netherlands,285446.340
EIRE,283453.960
Germany,228867.140
France,209715.110


In [41]:
# Unstancking the Description and Resetting the Index
grouped_df = df_new[df_new['Country'] == 'France'].groupby(['InvoiceNo', 'Description']).size().unstack().fillna(0).reset_index().set_index('InvoiceNo')
grouped_df.head()


Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# Defining a Custom Encoding
def custom_encoding(x):
  if x<=0:
    return 0
  elif x>=1:
    return 1
grouped_df = grouped_df.applymap(custom_encoding)
grouped_df.drop('POSTAGE', inplace = True, axis =1)

In [43]:
# Market Basket Analysis
frequent_items = apriori(grouped_df, min_support = 0.05, use_colnames= True)

In [44]:
# Rules
rules = association_rules(frequent_items, metric= 'confidence', min_threshold=0.6)

In [45]:
# Sort
rules.sort_values(by = 'lift', ascending= False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
10,(PACK OF 6 SKULL PAPER CUPS),(PACK OF 6 SKULL PAPER PLATES),0.063776,0.056122,0.051020,0.800000,14.254545,1.0,0.047441,4.719388,0.993188,0.740741,0.788108,0.854545
11,(PACK OF 6 SKULL PAPER PLATES),(PACK OF 6 SKULL PAPER CUPS),0.056122,0.063776,0.051020,0.909091,14.254545,1.0,0.047441,10.298469,0.985135,0.740741,0.902898,0.854545
7,(CHILDRENS CUTLERY DOLLY GIRL),(CHILDRENS CUTLERY SPACEBOY),0.071429,0.068878,0.063776,0.892857,12.962963,1.0,0.058856,8.690476,0.993846,0.833333,0.884932,0.909392
6,(CHILDRENS CUTLERY SPACEBOY),(CHILDRENS CUTLERY DOLLY GIRL),0.068878,0.071429,0.063776,0.925926,12.962963,1.0,0.058856,12.535714,0.991123,0.833333,0.920228,0.909392
25,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",(ALARM CLOCK BAKELIKE RED),0.073980,0.094388,0.063776,0.862069,9.133271,1.0,0.056793,6.565689,0.961653,0.609756,0.847693,0.768872
26,(ALARM CLOCK BAKELIKE RED),"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",0.094388,0.073980,0.063776,0.675676,9.133271,1.0,0.056793,2.855230,0.983324,0.609756,0.649765,0.768872
24,"(ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...",(ALARM CLOCK BAKELIKE GREEN),0.073980,0.096939,0.063776,0.862069,8.892922,1.0,0.056604,6.547194,0.958457,0.595238,0.847263,0.759982
27,(ALARM CLOCK BAKELIKE GREEN),"(ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...",0.096939,0.073980,0.063776,0.657895,8.892922,1.0,0.056604,2.706829,0.982825,0.595238,0.630564,0.759982
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,1.0,0.069932,5.568878,0.976465,0.704545,0.820431,0.826814
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,1.0,0.069932,4.916181,0.979224,0.704545,0.796590,0.826814


OMNI CHANNEL RETAIL DATA

In [52]:
# Open the key file
with open('file_key.key', 'rb') as f:
  key = f.read()

In [53]:
# Store it in fernet
fernet = Fernet(key)

In [54]:
# Open the data
with open('retail_data.csv', 'rb') as f:
  data = f.read()

In [55]:
# Decrypt the data
decryption = fernet.decrypt(data)

In [56]:
# Overwrite the data with Decryption
with open('retail_data.csv', 'wb') as f:
  f.write(decryption)

In [57]:
# Load the data with pandas
retail = pd.read_csv('retail_data.csv')
retail.head()

,visit_id,customer_id,visit_date,channel,dwell_seconds,is_purchase,items,num_items,total_amount,total_cogs,return_flag,fulfillment_status,satisfaction,first_visit_date,is_new_customer,acquisition_channel,Category
0,V100029,C0297,2025-06-01 02:35:05,E-Commerce,106,True,Chips (x1),1,40.0,23.86,True,Fulfilled,2.0,2025-06-01 02:35:05,True,E-Commerce,Electronics
1,V100255,C0025,2025-06-01 07:26:34,Store,151,True,Floor Cleaner (x1),1,180.0,115.54,False,In-Store Pickup/Immediate,3.0,2025-06-01 07:26:34,True,Store,Stationery
2,V100057,C0087,2025-06-01 12:08:39,E-Commerce,710,False,NaN,0,0.0,0.00,False,NaN,NaN,2025-06-01 12:08:39,True,E-Commerce,NaN
3,V100472,C0046,2025-06-02 00:36:32,E-Commerce,62,False,NaN,0,0.0,0.00,False,NaN,NaN,2025-06-02 00:36:32,True,E-Commerce,NaN
4,V100454,C0169,2025-06-02 07:42:42,App,44,False,NaN,0,0.0,0.00,False,NaN,NaN,2025-06-02 07:42:42,True,App,NaN


In [58]:
# Missing Values
retail.isna().sum()

,0
visit_id,0
customer_id,0
visit_date,0
channel,0
dwell_seconds,0
is_purchase,0
items,391
num_items,0
total_amount,0
total_cogs,0


In [59]:
# Missing Data
missing_rows = retail[retail.isna().any(axis=1)]
all_missing_due_to_no_purchase = (missing_rows['is_purchase'] == 0).all()
print("Are all missing values only where no purchase happened? :", all_missing_due_to_no_purchase)

Are all missing values only where no purchase happened? : True


In [60]:
# Datatypes
retail.dtypes

,0
visit_id,object
customer_id,object
visit_date,object
channel,object
dwell_seconds,int64
is_purchase,bool
items,object
num_items,int64
total_amount,float64
total_cogs,float64


In [61]:
# make the copy of the data
retail_copy = retail.copy(deep = True)
retail_copy.head()

,visit_id,customer_id,visit_date,channel,dwell_seconds,is_purchase,items,num_items,total_amount,total_cogs,return_flag,fulfillment_status,satisfaction,first_visit_date,is_new_customer,acquisition_channel,Category
0,V100029,C0297,2025-06-01 02:35:05,E-Commerce,106,True,Chips (x1),1,40.0,23.86,True,Fulfilled,2.0,2025-06-01 02:35:05,True,E-Commerce,Electronics
1,V100255,C0025,2025-06-01 07:26:34,Store,151,True,Floor Cleaner (x1),1,180.0,115.54,False,In-Store Pickup/Immediate,3.0,2025-06-01 07:26:34,True,Store,Stationery
2,V100057,C0087,2025-06-01 12:08:39,E-Commerce,710,False,NaN,0,0.0,0.00,False,NaN,NaN,2025-06-01 12:08:39,True,E-Commerce,NaN
3,V100472,C0046,2025-06-02 00:36:32,E-Commerce,62,False,NaN,0,0.0,0.00,False,NaN,NaN,2025-06-02 00:36:32,True,E-Commerce,NaN
4,V100454,C0169,2025-06-02 07:42:42,App,44,False,NaN,0,0.0,0.00,False,NaN,NaN,2025-06-02 07:42:42,True,App,NaN


In [62]:
# Create a new column as Date
retail_copy['Date'] = pd.to_datetime(retail_copy['visit_date'])

In [66]:
# Customer Footfall
footfalls = retail_copy.groupby(retail_copy['Date'].dt.date)['customer_id'].nunique().sort_values(ascending = False)
print(footfalls)

Date
2025-08-18    11
2025-07-29    11
2025-07-30    10
2025-08-09    10
2025-08-29     8
              ..
2025-06-26     1
2025-06-19     1
2025-08-06     1
2025-09-02     1
2025-09-03     1
Name: customer_id, Length: 106, dtype: int64


In [70]:
# Average Dwell Time
avg_dwell = retail_copy.groupby('customer_id')['dwell_seconds'].mean()
round(avg_dwell.mean(),2)

np.float64(268.11)

In [72]:
# Store layout effectiveness
layout_eff = retail_copy.groupby('Category')['dwell_seconds'].mean().sort_values(ascending = False)
layout_eff

,dwell_seconds
Category,
Grocery,376.187500
Apparel,321.428571
Cosmetics,296.954545
Electronics,274.545455
Stationery,274.380952


In [73]:
# Aggregate tutal spent per customer
customer_spend = retail_copy.groupby('customer_id')['total_amount'].sum().reset_index()
customer_spend.rename(columns = {'total_amount': 'total_spend'}, inplace = True)

In [74]:
# Percentile Based Spend Score
retail_copy['spend_score'] = (customer_spend['total_spend'] - customer_spend['total_spend'].min()/
                              customer_spend['total_spend'].max() - customer_spend['total_spend'].min()).round(2)

In [75]:
# See the Spend Score
retail_copy['spend_score'].head()

,spend_score
0,0.0
1,0.0
2,3116.0
3,0.0
4,299.0


In [77]:
# Buying Behaviour Classification
def classify_behavior(row):
    if row['spend_score'] > 70 and row['dwell_seconds'] > 40:
        return 'Complex Buying'
    elif row['spend_score'] > 70 and row['dwell_seconds'] <= 40:
        return 'Impulsive Buying'
    elif row['spend_score'] < 40 and row['dwell_seconds'] < 20:
        return 'Habitual Buying'
    else:
        return 'Variety Seeking Buying'

retail_copy['Buying_Behavior'] = retail_copy.apply(classify_behavior, axis=1)

print("Buying Behavior Distribution:")
print(retail_copy['Buying_Behavior'].value_counts(normalize=True) * 100)

Buying Behavior Distribution:
Buying_Behavior
Variety Seeking Buying    83.6
Complex Buying            15.4
Impulsive Buying           1.0
Name: proportion, dtype: float64


In [80]:
# Channel Footfall
channel_footfall = retail_copy.groupby('channel')['customer_id'].nunique().sort_values(ascending = False)
channel_footfall

,customer_id
channel,
Store,153
E-Commerce,130
App,99


In [82]:
# Conversion rate
conversion_rate = retail_copy.groupby("channel")["is_purchase"].mean().sort_values(ascending = False)
conversion_rate

,is_purchase
channel,
Store,0.240566
E-Commerce,0.206897
App,0.192982


In [84]:
# Average Order Value based on spend score
aov = retail_copy.groupby('channel')['spend_score'].mean().sort_values(ascending = False)
aov

,spend_score
channel,
E-Commerce,443.341772
Store,137.634783
App,103.283333


In [85]:
# Return Rate
retail_copy["return_flag"] = retail_copy["return_flag"].astype(int)

return_rate = retail_copy.groupby("channel")["return_flag"].mean() * 100
print("Return Rate by Channel (%):")
print(return_rate.round(2))

Return Rate by Channel (%):
channel
App           1.75
E-Commerce    1.72
Store         0.47
Name: return_flag, dtype: float64


In [86]:
# Omni-channel fulfillment rate (proxy: customers buying in >1 channel)
multi_channel_customers = retail_copy.groupby('customer_id')['channel'].nunique()
omni_rate = (multi_channel_customers  > 1 ).mean()
print('Omni_Channel Fulfillment Rate is', round(omni_rate,2))

Omni_Channel Fulfillment Rate is 0.43


In [89]:
# Average Customer Satisfaction
if 'satisfaction' in retail_copy.columns:
    satisfaction = retail_copy.groupby('channel')['satisfaction'].mean().sort_values(ascending = False)
    print(satisfaction)
else:
    satisfaction = "Not available"
    print(satisfaction)

channel
Store         4.176471
E-Commerce    4.111111
App           4.045455
Name: satisfaction, dtype: float64
